In [ ]:
pip install langgraph langchain-openai langchain-community python-docx python-dotenv

In [ ]:
from typing import TypedDict, List, Dict, Optional
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langgraph.graph import StateGraph, END
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI  # or your preferred LLM
from dotenv import load_dotenv
import os

In [ ]:

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [6]:
# ==================
# 1. Define State
# ==================
class AgentState(TypedDict):
    messages: List[Dict[str, str]]  # Conversation history
    symptoms_summary: Optional[str]
    report_text: Optional[str]
    verification_answers: Dict[str, str]
    follow_up_tasks: List[str]

In [ ]:
# ==================
# 2. Initialize LLM
# ==================
llm = ChatOpenAI(temperature=0.3, model="gpt-4o-mini")

In [11]:
# ==================
# 3. Define Agents
# ==================
def symptom_collector_node(state: AgentState):
    """Agent for symptom collection conversation"""
    memory = ConversationBufferMemory()
    
    # Load previous conversation
    for msg in state["messages"]:
        if msg["type"] == "human":
            memory.chat_memory.add_user_message(msg["content"])
        else:
            memory.chat_memory.add_ai_message(msg["content"])
    
    # Create structured interview prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a medical assistant conducting a patient interview. 
         Ask one question at a time about symptoms, duration, severity, and medications.
         When finished, create a structured summary.""")
    ])
    
    chain = prompt | llm
    response = chain.invoke({"input": memory.load_memory_variables({})})
    
    # Update state
    state["messages"].append({"type": "ai", "content": response.content})
    
    # If summary is detected, finalize
    if "SUMMARY:" in response.content:
        state["symptoms_summary"] = response.content.split("SUMMARY:")[-1].strip()
    
    return state


In [12]:
def test_report_processor_node(state: AgentState):
    """Process DOCX reports and handle Q&A"""
    if "report.docx" not in state.get("uploaded_files", []):
        return state
    
    # Load DOCX
    loader = Docx2txtLoader("report.docx")
    docs = loader.load()
    state["report_text"] = "\n".join([doc.page_content for doc in docs])
    
    # Handle Q&A
    last_message = state["messages"][-1]["content"]
    
    qa_prompt = ChatPromptTemplate.from_template("""
    Medical Report:
    {report}
    
    Question: {question}
    Answer concisely and professionally:
    """)
    
    qa_chain = qa_prompt | llm
    answer = qa_chain.invoke({
        "report": state["report_text"],
        "question": last_message
    })
    
    state["messages"].append({"type": "ai", "content": answer.content})
    return state

In [13]:
def verification_question_generator_node(state: AgentState):
    """Generate verification questions from report"""
    if not state.get("report_text"):
        return state
    
    prompt = ChatPromptTemplate.from_template("""
    Based on these test findings:
    {report}
    
    Generate as many verification questions to confirm the patient actually experiences 
    the symptoms mentioned in the report. Use layman's terms.
    
    Format: 
    1. Question 1
    2. Question 2
    3. Question 3
    4. continue
    """)
    
    chain = prompt | llm
    questions = chain.invoke({"report": state["report_text"]})
    
    state["messages"].append({
        "type": "ai",
        "content": f"Verification Questions:\n{questions.content}"
    })
    return state


In [14]:
def follow_up_reporter_node(state: AgentState):
    """Handle follow-ups and doctor reporting"""
    prompt = ChatPromptTemplate.from_template("""
    Patient Summary:
    {summary}
    
    Test Report Findings:
    {report}
    
    Verification Answers:
    {answers}
    
    Create a final doctor report with:
    1. Key symptoms
    2. Test correlations
    3. Urgency level
    4. Recommended next steps
    """)
    
    chain = prompt | llm
    report = chain.invoke({
        "summary": state["symptoms_summary"],
        "report": state["report_text"],
        "answers": state["verification_answers"]
    })
    
    state["follow_up_tasks"] = [
        "Schedule follow-up in 3 days",
        "Send report to Dr. Smith",
        "Check medication compatibility"
    ]
    
    state["messages"].append({
        "type": "ai",
        "content": f"Doctor Report:\n{report.content}"
    })
    return state


In [19]:
# ==================
# 4. Build Workflow
# ==================
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("symptom_collector", symptom_collector_node)
workflow.add_node("test_report_processor", test_report_processor_node)
workflow.add_node("verification_questions", verification_question_generator_node)
workflow.add_node("follow_up_reporter", follow_up_reporter_node)

# Set up edges
workflow.set_entry_point("symptom_collector")

workflow.add_edge("symptom_collector", "test_report_processor")
# After (correct)
workflow.add_conditional_edges(
    "test_report_processor",
    lambda state: "yes" if "report.docx" in state.get("uploaded_files", []) else "no",
    {"yes": "verification_questions", "no": "follow_up_reporter"}
)
workflow.add_edge("verification_questions", "follow_up_reporter")
workflow.add_edge("follow_up_reporter", END)

# ==================
# 5. Compile Agent
# ==================
medical_agent = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(medical_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    # This requires some extra dependencies and is optional
    print(e)

In [ ]:
# ==================
# 6. Usage Example
# ==================
if __name__ == "__main__":
    initial_state = {
        "messages": [{"type": "human", "content": "I'm having chest pain"}],
        "symptoms_summary": None,
        "report_text": None,
        "verification_answers": {},
        "follow_up_tasks": [],
        "uploaded_files": ["report.docx"]  # Remove if no report
    }
    
    for step in medical_agent.stream(initial_state):
        node, new_state = next(iter(step.items()))
        print(f"=== {node} ===")
        print(new_state["messages"][-1]["content"])
        print("\n---\n")